In [1]:
import numpy as np
import torch.nn as nn
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import sys
sys.path.append('../')
from nmt.encoder import NMTEncoder

$\textbf{Examine nn.Embedding}:$

Create a LongTensor A with a shape (3, 4): (batch_size, seq_size)

In [2]:
torch.manual_seed(7)
A = torch.LongTensor([[1,2,3,1], [3,1,1,0], [3,2,0,0]])
A

tensor([[1, 2, 3, 1],
        [3, 1, 1, 0],
        [3, 2, 0, 0]])

In [3]:
torch.manual_seed(7)
em = nn.Embedding(4, 5, padding_idx=0)  #(vocab_size, embedding_size)
em(A)

tensor([[[ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
         [-0.0810,  1.3615, -2.0615,  0.6741, -1.3233],
         [-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
         [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169]],

        [[-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
         [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
         [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
         [-0.0810,  1.3615, -2.0615,  0.6741, -1.3233],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<EmbeddingBackward>)

In [4]:
torch.manual_seed(7)
test = NMTEncoder(4, 5, 7, 1)
test.source_embedding(A)

/disk/ocean/lhe/conda/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[[ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
         [-0.0810,  1.3615, -2.0615,  0.6741, -1.3233],
         [-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
         [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169]],

        [[-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
         [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
         [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
         [-0.0810,  1.3615, -2.0615,  0.6741, -1.3233],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<EmbeddingBackward>)

In [5]:
L = torch.Tensor([4, 3, 2])
L

tensor([4., 3., 2.])

In [6]:
x_packed = pack_padded_sequence(test.source_embedding(A), L.detach().cpu().numpy(), 
                                        batch_first=True)
x_packed

PackedSequence(data=tensor([[ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
        [-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
        [-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
        [-0.0810,  1.3615, -2.0615,  0.6741, -1.3233],
        [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
        [-0.0810,  1.3615, -2.0615,  0.6741, -1.3233],
        [-1.3598, -0.8667, -0.5640, -2.0565, -0.3567],
        [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169],
        [ 0.7821,  1.0391, -0.7245, -0.8489, -1.2169]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([3, 3, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [7]:
x_birnn_out, x_birnn_h  = test.birnn(x_packed)

In [8]:
x_birnn_out.data.shape

torch.Size([9, 14])

In [9]:
print(x_birnn_out.data.shape) # (num_items (非零项), num_rnn * hidden_size)
x_birnn_out

torch.Size([9, 14])


PackedSequence(data=tensor([[-0.0257,  0.3514, -0.1862,  0.0716, -0.1423, -0.3681, -0.1023, -0.0204,
         -0.6083, -0.2706, -0.2377, -0.4160, -0.3308, -0.7484],
        [-0.3060, -0.1342,  0.0226, -0.1534,  0.0420, -0.3409, -0.0047,  0.3069,
         -0.6709, -0.4534,  0.0832, -0.2172,  0.0017, -0.4740],
        [-0.3060, -0.1342,  0.0226, -0.1534,  0.0420, -0.3409, -0.0047, -0.0758,
         -0.6710, -0.2347,  0.4194, -0.1586, -0.2441, -0.3111],
        [ 0.1014,  0.5060, -0.1416,  0.3485, -0.2273, -0.4966, -0.2600, -0.2441,
         -0.7337,  0.1883,  0.2000, -0.2370, -0.4817, -0.5861],
        [-0.1428,  0.2943, -0.1686,  0.0132, -0.1000, -0.5351, -0.0853,  0.2300,
         -0.5012, -0.3525, -0.3890, -0.5313, -0.1995, -0.6894],
        [-0.0278,  0.3530,  0.0063,  0.2001, -0.0558, -0.4587, -0.2121, -0.2664,
         -0.5863,  0.2823,  0.1455, -0.3005, -0.4738, -0.3848],
        [-0.2679,  0.1561, -0.0934, -0.1150, -0.1729, -0.6172, -0.1933,  0.2854,
         -0.6024, -0.4073,  0

In [15]:
x_birnn_h = x_birnn_h.permute(1, 0, 2)
x_birnn_h

tensor([[[-0.1303,  0.4127, -0.2643,  0.0518, -0.2694, -0.7116, -0.1506],
         [-0.0827,  0.4723, -0.3154,  0.1116, -0.2272, -0.6654, -0.1218],
         [-0.0278,  0.3530,  0.0063,  0.2001, -0.0558, -0.4587, -0.2121]],

        [[-0.0204, -0.6083, -0.2706, -0.2377, -0.4160, -0.3308, -0.7484],
         [ 0.3069, -0.6709, -0.4534,  0.0832, -0.2172,  0.0017, -0.4740],
         [-0.0758, -0.6710, -0.2347,  0.4194, -0.1586, -0.2441, -0.3111]]],
       grad_fn=<PermuteBackward>)

In [11]:
x_unpacked, _ = pad_packed_sequence(x_birnn_out, batch_first=True)
x_unpacked.shape  # (batch_size, seq_length, num_rnn=2 * hidden_size)

torch.Size([3, 4, 14])

In [12]:
x_unpacked

tensor([[[-0.0257,  0.3514, -0.1862,  0.0716, -0.1423, -0.3681, -0.1023,
          -0.0204, -0.6083, -0.2706, -0.2377, -0.4160, -0.3308, -0.7484],
         [ 0.1014,  0.5060, -0.1416,  0.3485, -0.2273, -0.4966, -0.2600,
          -0.2441, -0.7337,  0.1883,  0.2000, -0.2370, -0.4817, -0.5861],
         [-0.2679,  0.1561, -0.0934, -0.1150, -0.1729, -0.6172, -0.1933,
           0.2854, -0.6024, -0.4073,  0.1458, -0.1950,  0.0063, -0.3204],
         [-0.1303,  0.4127, -0.2643,  0.0518, -0.2694, -0.7116, -0.1506,
           0.1844, -0.3343, -0.2468, -0.2973, -0.4267, -0.1635, -0.4404]],

        [[-0.3060, -0.1342,  0.0226, -0.1534,  0.0420, -0.3409, -0.0047,
           0.3069, -0.6709, -0.4534,  0.0832, -0.2172,  0.0017, -0.4740],
         [-0.1428,  0.2943, -0.1686,  0.0132, -0.1000, -0.5351, -0.0853,
           0.2300, -0.5012, -0.3525, -0.3890, -0.5313, -0.1995, -0.6894],
         [-0.0827,  0.4723, -0.3154,  0.1116, -0.2272, -0.6654, -0.1218,
           0.1844, -0.3343, -0.2468, -0.297

In [13]:
x_birnn_h.contiguous().view(x_birnn_h.size(0), -1)

tensor([[-0.1303,  0.4127, -0.2643,  0.0518, -0.2694, -0.7116, -0.1506, -0.0204,
         -0.6083, -0.2706, -0.2377, -0.4160, -0.3308, -0.7484],
        [-0.0827,  0.4723, -0.3154,  0.1116, -0.2272, -0.6654, -0.1218,  0.3069,
         -0.6709, -0.4534,  0.0832, -0.2172,  0.0017, -0.4740],
        [-0.0278,  0.3530,  0.0063,  0.2001, -0.0558, -0.4587, -0.2121, -0.0758,
         -0.6710, -0.2347,  0.4194, -0.1586, -0.2441, -0.3111]],
       grad_fn=<ViewBackward>)

Notice that x_birnn_h is equal to concat each last $\textbf{non-trivial}$ state of x_unpacked!

In [14]:
test.forward(A, L)

(tensor([[[-0.0257,  0.3514, -0.1862,  0.0716, -0.1423, -0.3681, -0.1023,
           -0.0204, -0.6083, -0.2706, -0.2377, -0.4160, -0.3308, -0.7484],
          [ 0.1014,  0.5060, -0.1416,  0.3485, -0.2273, -0.4966, -0.2600,
           -0.2441, -0.7337,  0.1883,  0.2000, -0.2370, -0.4817, -0.5861],
          [-0.2679,  0.1561, -0.0934, -0.1150, -0.1729, -0.6172, -0.1933,
            0.2854, -0.6024, -0.4073,  0.1458, -0.1950,  0.0063, -0.3204],
          [-0.1303,  0.4127, -0.2643,  0.0518, -0.2694, -0.7116, -0.1506,
            0.1844, -0.3343, -0.2468, -0.2973, -0.4267, -0.1635, -0.4404]],
 
         [[-0.3060, -0.1342,  0.0226, -0.1534,  0.0420, -0.3409, -0.0047,
            0.3069, -0.6709, -0.4534,  0.0832, -0.2172,  0.0017, -0.4740],
          [-0.1428,  0.2943, -0.1686,  0.0132, -0.1000, -0.5351, -0.0853,
            0.2300, -0.5012, -0.3525, -0.3890, -0.5313, -0.1995, -0.6894],
          [-0.0827,  0.4723, -0.3154,  0.1116, -0.2272, -0.6654, -0.1218,
            0.1844, -0.3343, 